<hr style="height: 4px; color: #839D2D; width: 100%; ">

# <font color='#61210B'>Formation OpenClassRooms   -   Parcours DATA ANALYST</font>

<hr style="height: 2px; color: #839D2D; width: 100%; ">

## <font color='#38610B'>Projet V - Etude de Marché Internationale</font>

### Partie 1 - Constitution Jeu de Données & Nettoyage 
<u>Les données sont issues des sites web suivants</u> :
- Organisation des Nations Unies pour l'alimentation et l'agriculture (<a href="http://www.fao.org/faostat/en/#data">FAO</a>)  
- Banque Mondiale - World Bank Open Data (<a href="https://databank.banquemondiale.org/data/home.aspx">WBD</a>)  
- Wikipedia (données ISO-3166 Codes pays)

<u>DataFrames pandas utilisés et critères de téléchargement</u> :  
* <font color='#8A0808'>DataFrame <strong>pays</strong></font> : Table de correspondance entre les codes pays issus de la FAO, de la WBD, et la norme Iso-3166  
Constitué avec Excel en consolidant toutes les données pays issus de la FAO, WBD et de Wikipedia pour la norme ISO-3166  


* <font color='#8A0808'>DataFrame <strong>fao</strong></font> : Année 2013 - Population & Bilans Alimentaires Volaille & Viande (source FAO)   
Critères de selection : (**Pays** = tous, **Eléments** = tous, **Année** = 2013, **Produits** = Volailles, **Groupe Produits** = Viande (total) )  


* <font color='#8A0808'>DataFrame <strong>wbdxch</strong></font> : Année 2013 - Achat Vente de Volailles entre France & Autres Pays (en $)   
(**Pays Acheteur** = tous, **Pays Vendeur** = France, **Année** = 2013, **Produits** = Volailles vivante & Viande Volailles )


* <font color='#8A0808'>DataFrame <strong>wbdeco</strong></font> Année 2013 - Données Macro-economiques (source WBD)

  
* <font color='#013ADF'>DataFrame <strong>gen</strong></font> : Fichier enrichi, nettoyé pour etude de marché


**<font color='#38610B'>- Date : 15 Avr 2019</font>**  
Auteur : Frédéric Boissy
<hr style="height: 4px; color: #839D2D; width: 100%; ">


In [1]:
# -*- coding: utf8 -*-
import numpy as np
import pandas as pd
pd.options.display.float_format = '{:,.2f}'.format   # Nombres avec sepa milliers "," et 2décimales après "."
pd.options.mode.use_inf_as_na = True

import seaborn as sns
import matplotlib as matplt
import matplotlib.pyplot as plt
import scipy as sc
import scipy.stats as scst
from scipy.cluster.hierarchy import linkage, fcluster, dendrogram
import statsmodels as st
from sklearn import decomposition, preprocessing
from functions import *

from IPython.display import display, Markdown, HTML  # pour gérer un affichage plus joli que la fonction "print"

import time   # Librairie temps pour calculs durée par exemple
trt_start_time = time.time()

In [2]:
pays = pd.read_excel("DATA/0-Table_Corresp_Pays_ISO_FAO_WBD.xlsx", sheet_name='data_pj5') 
fao = pd.read_excel("DATA/1-FAO_Data_Principales.xlsx", sheet_name='data_pj5') 
wbdxch = pd.read_excel("DATA/2-WBD_Data_CNUCED_FRA_2013.xlsx", sheet_name='data_pj5')
wbdeco = pd.read_excel("DATA/3-WBD_Data_Indic_Dev_PIB_2013.xlsx", sheet_name='data_pj5')

<hr style="height: 3px; color: #839D2D; width: 100%; ">

###  <font color='#61210B'><u>Table de Correspondance Code Pays</u></font> : Dataframe 'pays'
#### Détails des zones

- <b>cpays_fao :</b> Code Pays (Source FAO)   
- <b>pays_fao :</b> Nom Pays (Source FAO)   
- <b>cpays_wbd :</b> Code Pays (Source WBD)   
- <b>pays_wbd :</b> Nom Pays (Source WBD)   
- <b>cpays_iso_n2 :</b> Code Pays Norme ISO - 2 digits numeriques (Source Wikipedia)   
- <b>cpays_iso_a2 :</b> Code Pays Norme ISO - 2 digits alphanum (Source Wikipedia)   
- <b>cpays_iso_a3 :</b> Code Pays Norme ISO - 3 digits alphanum (Source Wikipedia)   
- <b>pays_iso :</b> Nom Pays Norme ISO (Source Wikipedia)   
- <b>pays_iso_nom_francais :</b> Nom Pays Français Norme ISO (Source Wikipedia)   

In [3]:
pays.head()

,cpays_fao,pays_fao,cpays_wbd,pays_wbd,cpays_iso_n2,cpays_iso_a2,cpays_iso_a3,pays_iso,pays_iso_nom_francais
0,1.00,Armenie,ARM,Armenie,51,AM,ARM,Arménie,Armenie
1,2.00,Afghanistan,AFG,Afghanistan,4,AF,AFG,Afghanistan,Afghanistan
2,3.00,Albanie,ALB,Albanie,8,AL,ALB,Albanie,Albanie
3,4.00,Algerie,DZA,Algerie,12,DZ,DZA,Algérie,Algerie
4,5.00,Samoa americaines,ASM,Samoa americaines,16,AS,ASM,Samoa Américaines,Samoa Americaines


Pas de nettoyage ici, table brute pour ne perdre aucune informations.

<hr style="height: 3px; color: #839D2D; width: 100%; ">

###  <font color='#61210B'><u>Données Requises Projets</u></font> : Dataframe 'fao'
#### Détails des zones

- <b>cpays_fao :</b> Code Pays   
- <b>pays_fao :</b> nom Pays   
- <b>pop_2010 :</b> Population en 2010   
- <b>pop_2013 :</b> Population en 2013   
- <b>evo_2010_2013 :</b> (%) Evolution population entre 2010 & 2013   
- <b>qt_disp_(kcal/p/j) :</b> Quantité dispo alimentaire totale pays (en Kcal/pers/jour)   
- <b>qt_disp_prot_(gr/p/j) :</b> Quantité dispo alim. protéines totale pays (en Grammes/pers/jour)   
- <b>qt_prot_via_(gr/p/j) :</b> Quantité dispo alim. protéines viande (volaille incluse) pays (en Grammes/pers/jour)   
- <b>qt_prot_vol_(gr/p/j) :</b> Quantité dispo alim. protéines volaille pays (en Grammes/pers/jour)   
- <b>ratio_prot_viande :</b> Ratio Qté Proteines Viande (volaille incluse) / Qté Proteines Totale du Pays   
- <b>ratio_prot_volaille :</b> Ratio Qté Proteines Volaille / Qté Proteines Totale du Pays   
- <b>qt_exp_via_(t) :</b> Quantité Export viande (volaille incluse) pays (en tonnes/an)   
- <b>qt_exp_vol_(t) :</b> Quantité Export volaille pays (en tonnes/an)   
- <b>qt_imp_via_(t) :</b> Quantité Import viande (volaille incluse) pays (en tonnes/an)   
- <b>qt_imp_vol_(t) :</b> Quantité Import volaille pays (en tonnes/an)   
- <b>qt_prod_via_(t) :</b> Quantité Production viande (volaille incluse) pays (en tonnes/an)   
- <b>qt_prod_vol_(t) :</b> Quantité Production volaille pays (en tonnes/an)   


In [4]:
fao.describe(include='all')

,cpays_fao,pays_fao,pop_2010,pop_2013,evo_2010_2013,qt_disp_(kcal/p/j),qt_disp_prot_(gr/p/j),qt_prot_via_(gr/p/j),qt_prot_vol_(gr/p/j),ratio_prot_viande,ratio_prot_volaille,qt_exp_via_(t),qt_exp_vol_(t),qt_imp_via_(t),qt_imp_vol_(t),qt_prod_via_(t),qt_prod_vol_(t)
count,231.00,231,229.00,229.00,227.00,174.00,174.00,174.00,174.00,174.00,174.00,164.00,147.00,174.00,173.00,174.00,172.00
unique,nan,231,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
top,nan,Tokelaou,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
freq,nan,1,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
mean,131.35,NaN,"30,367,259.08","31,479,462.74",4.07,"2,848.94",81.37,17.49,7.14,20.31,8.29,"280,597.56","113,551.02","243,339.08","87,335.26","1,776,528.74","630,941.86"
std,75.22,NaN,"125,485,994.26","128,920,703.18",4.54,438.12,19.99,10.39,5.69,9.50,6.18,"921,642.51","487,157.69","530,645.48","194,877.16","7,399,835.59","2,315,308.27"
min,1.00,NaN,794.00,801.00,-6.53,"1,879.00",37.66,1.32,0.14,2.19,0.22,0.00,0.00,0.00,0.00,0.00,0.00
25%,67.50,NaN,"416,110.00","451,338.00",1.10,"2,531.50",64.39,9.20,1.97,12.48,3.36,0.00,0.00,"8,000.00","2,000.00","71,000.00","12,750.00"
50%,131.00,NaN,"5,074,252.00","5,366,277.00",3.56,"2,833.00",81.59,16.41,6.48,20.69,7.43,"6,500.00","1,000.00","44,500.00","16,000.00","255,000.00","59,500.00"
75%,194.50,NaN,"20,198,353.00","20,068,201.00",6.80,"3,189.50",94.95,25.13,10.68,27.36,11.71,"56,750.00","24,000.00","196,000.00","73,000.00","917,500.00","327,750.00"


#### Nettoyage & mise en Forme

In [5]:
# cas du Soudan => Après 2010, le Soudan s'est divisé en deux parties.
fao[fao['pays_fao'].str.contains('oudan', na=False) == True ]

,cpays_fao,pays_fao,pop_2010,pop_2013,evo_2010_2013,qt_disp_(kcal/p/j),qt_disp_prot_(gr/p/j),qt_prot_via_(gr/p/j),qt_prot_vol_(gr/p/j),ratio_prot_viande,ratio_prot_volaille,qt_exp_via_(t),qt_exp_vol_(t),qt_imp_via_(t),qt_imp_vol_(t),qt_prod_via_(t),qt_prod_vol_(t)
183,206,Soudan (ex),"44,453,155.00",nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
229,276,Soudan,nan,"36,849,918.00",nan,"2,336.00",70.71,10.70,0.45,15.13,0.64,"8,000.00",0.00,"2,000.00","2,000.00","1,032,000.00","46,000.00"
230,277,Soudan du Sud,nan,"11,177,490.00",nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan


Choix de ne garder que les données de 2013.   
- Donc suppression de la ligne codePays=206
- Modification des valeurs de populations et d'évolution par application de regle de 3 avec valeur de 2010 (code 206)

In [6]:
pop1 = fao[fao['cpays_fao'] == 276]['pop_2013'].values
pop2 = fao[fao['cpays_fao'] == 277]['pop_2013'].values
r = pop1/pop2
fao.loc[(fao['cpays_fao'] == 277), 'pop_2010'] = fao.loc[(fao['cpays_fao'] == 206), 'pop_2010'].values / (r+1)
fao.loc[(fao['cpays_fao'] == 276), 'pop_2010'] = fao.loc[(fao['cpays_fao'] == 206), 'pop_2010'].values * r / (r+1)
fao.drop(fao[fao.cpays_fao == 206].index, inplace=True)
fao.loc[(fao['cpays_fao'] == 276), 'evo_2010_2013'] = ((fao[fao['cpays_fao'] == 276]['pop_2013'].values / fao[fao['cpays_fao'] == 276]['pop_2010'].values)-1)*100
fao.loc[(fao['cpays_fao'] == 277), 'evo_2010_2013'] = ((fao[fao['cpays_fao'] == 277]['pop_2013'].values / fao[fao['cpays_fao'] == 277]['pop_2010'].values)-1)*100

In [7]:
# cas du Soudan => Après 2010, le Soudan s'est divisé en deux parties.
fao[fao['pays_fao'].str.contains('oudan', na=False) == True ]

,cpays_fao,pays_fao,pop_2010,pop_2013,evo_2010_2013,qt_disp_(kcal/p/j),qt_disp_prot_(gr/p/j),qt_prot_via_(gr/p/j),qt_prot_vol_(gr/p/j),ratio_prot_viande,ratio_prot_volaille,qt_exp_via_(t),qt_exp_vol_(t),qt_imp_via_(t),qt_imp_vol_(t),qt_prod_via_(t),qt_prod_vol_(t)
229,276,Soudan,"34,107,506.21","36,849,918.00",8.04,"2,336.00",70.71,10.70,0.45,15.13,0.64,"8,000.00",0.00,"2,000.00","2,000.00","1,032,000.00","46,000.00"
230,277,Soudan du Sud,"10,345,648.79","11,177,490.00",8.04,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan


In [8]:
fao[fao['qt_disp_(kcal/p/j)'].isna() == True].head()

,cpays_fao,pays_fao,pop_2010,pop_2013,evo_2010_2013,qt_disp_(kcal/p/j),qt_disp_prot_(gr/p/j),qt_prot_via_(gr/p/j),qt_prot_vol_(gr/p/j),ratio_prot_viande,ratio_prot_volaille,qt_exp_via_(t),qt_exp_vol_(t),qt_imp_via_(t),qt_imp_vol_(t),qt_prod_via_(t),qt_prod_vol_(t)
4,5,Samoa americaines,"55,637.00","55,307.00",-0.59,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
5,6,Andorre,"84,449.00","80,788.00",-4.34,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
12,13,Bahrein,"1,240,862.00","1,315,411.00",6.01,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
16,18,Bhoutan,"727,641.00","764,961.00",5.13,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
20,22,Aruba,"101,669.00","103,187.00",1.49,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan


In [9]:
# Choix d'ecarter les pays n'ayant pas de valeurs renseignées sur cette variable imposée
fao = fao.dropna(subset=['qt_disp_(kcal/p/j)'])

In [10]:
fao.describe(include='all')

,cpays_fao,pays_fao,pop_2010,pop_2013,evo_2010_2013,qt_disp_(kcal/p/j),qt_disp_prot_(gr/p/j),qt_prot_via_(gr/p/j),qt_prot_vol_(gr/p/j),ratio_prot_viande,ratio_prot_volaille,qt_exp_via_(t),qt_exp_vol_(t),qt_imp_via_(t),qt_imp_vol_(t),qt_prod_via_(t),qt_prod_vol_(t)
count,174.00,174,174.00,174.00,174.00,174.00,174.00,174.00,174.00,174.00,174.00,164.00,147.00,174.00,173.00,174.00,172.00
unique,nan,174,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
top,nan,Namibie,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
freq,nan,1,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
mean,125.43,NaN,"39,086,939.10","40,493,134.02",4.39,"2,848.94",81.37,17.49,7.14,20.31,8.29,"280,597.56","113,551.02","243,339.08","87,335.26","1,776,528.74","630,941.86"
std,73.42,NaN,"142,840,744.67","146,738,502.44",4.26,438.12,19.99,10.39,5.69,9.50,6.18,"921,642.51","487,157.69","530,645.48","194,877.16","7,399,835.59","2,315,308.27"
min,1.00,NaN,"51,445.00","53,169.00",-4.39,"1,879.00",37.66,1.32,0.14,2.19,0.22,0.00,0.00,0.00,0.00,0.00,0.00
25%,63.75,NaN,"2,308,040.00","2,450,341.75",1.39,"2,531.50",64.39,9.20,1.97,12.48,3.36,0.00,0.00,"8,000.00","2,000.00","71,000.00","12,750.00"
50%,120.50,NaN,"9,031,086.50","9,195,778.00",3.89,"2,833.00",81.59,16.41,6.48,20.69,7.43,"6,500.00","1,000.00","44,500.00","16,000.00","255,000.00","59,500.00"
75%,187.25,NaN,"27,325,041.25","29,276,370.50",7.02,"3,189.50",94.95,25.13,10.68,27.36,11.71,"56,750.00","24,000.00","196,000.00","73,000.00","917,500.00","327,750.00"


Controle du cas de la Chine (vu Etude PJ3)  
- Données FAO :  Le Code Pays "351" est une agrégation des Codes Pays (41, 96, 128, 214)

In [11]:
fao[fao['pays_fao'].str.contains("hine")]

,cpays_fao,pays_fao,pop_2010,pop_2013,evo_2010_2013,qt_disp_(kcal/p/j),qt_disp_prot_(gr/p/j),qt_prot_via_(gr/p/j),qt_prot_vol_(gr/p/j),ratio_prot_viande,ratio_prot_volaille,qt_exp_via_(t),qt_exp_vol_(t),qt_imp_via_(t),qt_imp_vol_(t),qt_prod_via_(t),qt_prod_vol_(t)
35,41,"Chine, continentale","1,359,755,102.00","1,382,793,212.00",1.69,"3,108.00",98.02,19.52,4.26,19.91,4.35,"901,000.00","564,000.00","2,054,000.00","584,000.00","83,462,000.00","18,265,000.00"
81,96,Chine - RAS de Hong-Kong,"7,025,221.00","7,148,571.00",1.76,"3,290.00",129.18,49.55,19.58,38.36,15.16,"1,047,000.00","806,000.00","2,023,000.00","1,186,000.00","167,000.00","28,000.00"
112,128,Chine - RAS de Macao,"536,969.00","575,841.00",7.24,"2,915.00",94.84,35.84,12.04,37.79,12.70,0.00,0.00,"56,000.00","17,000.00","13,000.00","4,000.00"
191,214,"Chine, Taiwan Province de","23,102,406.00","23,339,858.00",1.03,"2,997.00",89.74,24.96,10.69,27.81,11.91,"12,000.00","7,000.00","289,000.00","116,000.00","1,538,000.00","642,000.00"


Pas de soucis ici dans notre DataSet

<hr style="height: 3px; color: #839D2D; width: 100%; ">

###  <font color='#61210B'><u>Données Complémentaires - Echanges Commerciaux InterPays</u></font> : Dataframe 'wbdxch'
#### Détails des zones

ICI POSTULAT ENTREPRISE FRANCAISE

- <b>cpays_vnd :</b> Code Pays Vendeur   
- <b>pays_vnd :</b> Nom Pays Vendeur   
- <b>cpays_ach :</b> Code Pays Acheteur   
- <b>pays_ach :</b> Nom Pays Acheteur   
- <b>live_poultry_tr_value :</b> Valeur Marchande de Volaille vivante (US Dollars) Année 2013   
- <b>poultry_meat_tr_value :</b> Valeur Marchande de Viande de Volaille (US Dollars) Année 2013   
- <b>tot_poultry_tr_value :</b> Valeur Marchande de Totale de Volaille (US Dollars) Année 2013   
- <b>cpays_fao :</b> Code Pays FAO   
- <b>cpays_wbd :</b> Code Pays WBD   


In [12]:
wbdxch.head()

,cpays_vnd,pays_vnd,cpays_ach,pays_ach,live_poultry_tr_value,poultry_meat_tr_value,tot_poultry_tr_value,cpays_fao,cpays_wbd
0,250,France,4,Afghanistan,0.00,0.00,0.00,2.00,AFG
1,250,France,8,Albania,"11,738.00","318,497.00","330,235.00",3.00,ALB
2,250,France,12,Algeria,0.00,0.00,0.00,4.00,DZA
3,250,France,24,Angola,0.00,"1,584,588.00","1,584,588.00",7.00,AGO
4,250,France,28,Antigua and Barbuda,0.00,0.00,0.00,8.00,ATG


In [13]:
wbdxch.describe(include='all')

,cpays_vnd,pays_vnd,cpays_ach,pays_ach,live_poultry_tr_value,poultry_meat_tr_value,tot_poultry_tr_value,cpays_fao,cpays_wbd
count,188.00,188,188.00,188,188.00,188.00,188.00,187.00,187
unique,nan,1,nan,188,nan,nan,nan,nan,187
top,nan,France,nan,Poland,nan,nan,nan,nan,HND
freq,nan,188,nan,1,nan,nan,nan,nan,1
mean,250.00,NaN,426.38,NaN,"266,024.82","3,802,083.06","4,068,107.88",127.43,NaN
std,0.00,NaN,258.15,NaN,"819,045.19","23,869,973.43","24,044,539.85",75.40,NaN
min,250.00,NaN,4.00,NaN,0.00,0.00,0.00,1.00,NaN
25%,250.00,NaN,195.00,NaN,0.00,0.00,0.00,58.50,NaN
50%,250.00,NaN,424.00,NaN,0.00,0.00,0.00,126.00,NaN
75%,250.00,NaN,649.25,NaN,"39,608.00","89,570.00","399,787.75",193.50,NaN


In [14]:
# On ne conserve que les données pour les code_pays de la FAO
wbdxch = wbdxch.dropna(subset=['cpays_fao'])

<hr style="height: 3px; color: #839D2D; width: 100%; ">

###  <font color='#61210B'><u>Données Complémentaires - Macro Economiques</u></font> : Dataframe 'wbdeco'
#### Détails des zones

- <b>cpays_wbd :</b> Code Pays (WBD)   
- <b>pays_wbd :</b> Nom Pays (WBD)   
- <b>PIB_h_2013($US) :</b> PIB / habitant année 2013 (en US Dollars)   
- <b>evo_PIB_h_(%annu)_2013 :</b> Evolution / Croissance du PIB en 2013 (%)   


In [15]:
wbdeco.head()

,cpays_wbd,pays_wbd,PIB_h_2013($US),evo_PIB_h_(%annu)_2013
0,AFG,Afghanistan,647.97,2.16
1,ZAF,Afrique du Sud,"6,819.06",1.02
2,ALB,Albanie,"4,413.08",1.19
3,DZA,Algérie,"5,471.12",0.70
4,DEU,Allemagne,"46,530.91",0.22


In [16]:
wbdeco.describe(include='all')

,cpays_wbd,pays_wbd,PIB_h_2013($US),evo_PIB_h_(%annu)_2013
count,264,264,250.00,248.00
unique,264,264,nan,nan
top,HUN,République tchèque,nan,nan
freq,1,1,nan,nan
mean,NaN,NaN,"16,263.57",1.93
std,NaN,NaN,"24,735.60",4.67
min,NaN,NaN,255.37,-36.83
25%,NaN,NaN,"1,963.83",0.35
50%,NaN,NaN,"6,590.17",2.08
75%,NaN,NaN,"17,864.01",3.72


In [17]:
# Choix : On ne conserve que les pays contenant des Informations PIB renseignées dans les deux colonnes
wbdeco = wbdeco.dropna()

<hr style="height: 3px; color: #839D2D; width: 100%; ">

###  <font color='#61210B'><u>Constitution du jeu de données Consolidé pour Analyse</u></font> : Dataframe 'gen'
#### Détails des zones

<u>A partir du Dataframe "fao" </u> :
- <b>pays_fao :</b> Nom Pays FAO   
- <b>evo_2010_2013 :</b> (%) Evolution population entre 2010 & 2013   
- <b>qt_disp_(kcal/p/j) :</b> Quantité dispo alimentaire totale pays (en Kcal/pers/jour)   
- <b>qt_disp_prot_(gr/p/j) :</b> Quantité dispo alim. protéines totale pays (en Grammes/pers/jour)   
- <b>ratio_prot_viande :</b> Ratio Qté Proteines Viande (volaille incluse) / Qté Proteines Totale du Pays   


<u>A partir du Dataframe "xch" </u> :
- <b>tot_poultry_tr_value :</b> Valeur Marchande de Totale de Volaille (US Dollars) Année 2013   

<u>A partir du Dataframe "eco" </u> :
- <b>PIB_h_2013($US) :</b> PIB / habitant année 2013 (en US Dollars)   

<u>Zone Calculée</u> :
- <b>qt_vol_(kg/h/an) :</b> Quantité Volailles Disponible (en Kg/Hab/An)   
Formule -->   qt_vol_(kg/h/an) = [qt_prod_vol_(t) + qt_imp_vol_(t) - qt_exp_vol_(t)] / pop_2013

qt_exp_vol_(t) : Quantité Export volaille pays (en tonnes/an)  
qt_imp_vol_(t) : Quantité Import volaille pays (en tonnes/an)  
qt_prod_vol_(t) : Quantité Production volaille pays (en tonnes/an)  

In [18]:
gen = fao[['cpays_fao', 'pays_fao', 'pop_2013', 'evo_2010_2013', 'qt_disp_(kcal/p/j)', 'qt_disp_prot_(gr/p/j)', 
           'ratio_prot_viande', 'qt_prod_vol_(t)', 'qt_imp_vol_(t)', 'qt_exp_vol_(t)']].copy()
# On remplace les Nan par O pour les colonnes qt_prod, qt_imp, qt_exp, car il faut conserver ces lignes
gen = gen.fillna(0)
# Création de la nouvelle colonne
gen['qt_vol_(kg/h/an)'] = (gen['qt_prod_vol_(t)'] + gen['qt_imp_vol_(t)'] - gen ['qt_exp_vol_(t)'])*1000 / gen['pop_2013']
# Suppression des colonnes inutiles (pop et qté utilisées pour le calcul)
gen.drop(columns=['qt_prod_vol_(t)', 'qt_imp_vol_(t)', 'qt_exp_vol_(t)'], inplace=True)
# Renommage des Colonnes
gen.head()

,cpays_fao,pays_fao,pop_2013,evo_2010_2013,qt_disp_(kcal/p/j),qt_disp_prot_(gr/p/j),ratio_prot_viande,qt_vol_(kg/h/an)
0,1,Armenie,"2,893,509.00",0.56,"2,928.00",90.04,17.91,13.82
1,2,Afghanistan,"31,731,688.00",10.17,"2,090.00",58.25,8.33,2.33
2,3,Albanie,"2,918,978.00",-0.73,"3,193.00",111.42,20.39,14.39
3,4,Algerie,"38,338,562.00",6.15,"3,296.00",91.96,8.16,7.59
6,7,Angola,"25,998,340.00",11.25,"2,473.00",57.26,21.73,14.31


In [19]:
# Jointure Table Pays pour Recupérer le code pays wbd dans la table de transco
temp = pd.merge(gen,pays[['cpays_fao','cpays_wbd']], on='cpays_fao', how='left')
# Jointure Table eco pour Recupérer les données PIB des pays en 2013
temp = pd.merge(temp,wbdeco[['cpays_wbd','PIB_h_2013($US)']], on='cpays_wbd', how='left')
# Jointure Table echange pour Recupérer les données valeurs de volailles echangees avec la france en 2013
temp = pd.merge(temp,wbdxch[['cpays_wbd', 'tot_poultry_tr_value']], on='cpays_wbd', how='left')
temp.head()

,cpays_fao,pays_fao,pop_2013,evo_2010_2013,qt_disp_(kcal/p/j),qt_disp_prot_(gr/p/j),ratio_prot_viande,qt_vol_(kg/h/an),cpays_wbd,PIB_h_2013($US),tot_poultry_tr_value
0,1,Armenie,"2,893,509.00",0.56,"2,928.00",90.04,17.91,13.82,ARM,"3,843.59","4,538.00"
1,2,Afghanistan,"31,731,688.00",10.17,"2,090.00",58.25,8.33,2.33,AFG,647.97,0.00
2,3,Albanie,"2,918,978.00",-0.73,"3,193.00",111.42,20.39,14.39,ALB,"4,413.08","330,235.00"
3,4,Algerie,"38,338,562.00",6.15,"3,296.00",91.96,8.16,7.59,DZA,"5,471.12",0.00
4,7,Angola,"25,998,340.00",11.25,"2,473.00",57.26,21.73,14.31,AGO,"5,258.41","1,584,588.00"


In [20]:
gen = temp.copy()
# On supprime les colonnes inutiles et on renomme les autres pour faire plus synthétique / propre
gen.drop(columns=['cpays_fao', 'cpays_wbd'], inplace=True)
gen.columns = ['pays', 'population', 'evo_pop_%', 'qt_kcal/h', 'qt_prot_gr/h', 'ratio_prot_viande_%', 'qt_volaille_Kg/h', 
               'PIBh_$', 'tot_poultry_exch_$']
gen.head()

,pays,population,evo_pop_%,qt_kcal/h,qt_prot_gr/h,ratio_prot_viande_%,qt_volaille_Kg/h,PIBh_$,tot_poultry_exch_$
0,Armenie,"2,893,509.00",0.56,"2,928.00",90.04,17.91,13.82,"3,843.59","4,538.00"
1,Afghanistan,"31,731,688.00",10.17,"2,090.00",58.25,8.33,2.33,647.97,0.00
2,Albanie,"2,918,978.00",-0.73,"3,193.00",111.42,20.39,14.39,"4,413.08","330,235.00"
3,Algerie,"38,338,562.00",6.15,"3,296.00",91.96,8.16,7.59,"5,471.12",0.00
4,Angola,"25,998,340.00",11.25,"2,473.00",57.26,21.73,14.31,"5,258.41","1,584,588.00"


In [21]:
# On conserve les lignes données nan our live_poultry et poultry_meat car ce ne sont pas des erreurs  (on force à = 0 )
gen['tot_poultry_exch_$'] = gen['tot_poultry_exch_$'].fillna(0)

# On peut supprimer de notre analyse les pays n'ayant pas d'informations de PIB
gen = gen.dropna(subset=['PIBh_$'])
gen.describe(include='all')

,pays,population,evo_pop_%,qt_kcal/h,qt_prot_gr/h,ratio_prot_viande_%,qt_volaille_Kg/h,PIBh_$,tot_poultry_exch_$
count,169,169.00,169.00,169.00,169.00,169.00,169.00,169.00,169.00
unique,169,nan,nan,nan,nan,nan,nan,nan,nan
top,Namibie,nan,nan,nan,nan,nan,nan,nan,nan
freq,1,nan,nan,nan,nan,nan,nan,nan,nan
mean,NaN,"41,396,730.06",4.44,"2,853.47",81.44,20.21,22.19,"14,918.37","4,390,682.83"
std,NaN,"148,796,597.33",4.31,440.15,20.10,9.43,18.79,"21,011.93","25,339,643.29"
min,NaN,"53,169.00",-4.39,"1,879.00",37.66,2.19,0.38,332.92,0.00
25%,NaN,"2,869,107.00",1.39,"2,529.00",64.38,12.61,6.39,"1,871.33",0.00
50%,NaN,"9,385,293.00",3.91,"2,830.00",81.58,20.68,19.06,"6,168.39",0.00
75%,NaN,"29,944,476.00",7.02,"3,228.00",94.99,27.12,30.07,"16,326.91","393,266.00"


In [22]:
# On supprime aussi la France de notre Jeu de données car nous partons du postulat "entreprise Française"
gen.drop(gen[gen.pays == 'France'].index, inplace=True)

<hr style="height: 3px; color: #839D2D; width: 100%; ">

###  <font color='#61210B'><u>Export du Dataframe Consolidé</u></font> : Dataframe 'gen'
#### Dans un fichier csv - Pour analyse

In [23]:
gen.to_csv('DATA/selection_pays.csv', sep=',', encoding='utf-8', index=False)

In [24]:
dureetotale = round(time.time() - trt_start_time, 5)
print("--- Durée TOTALE du Notebook PJ5 -1- Load&Clean --- ", "%s seconds" % dureetotale)

--- Durée TOTALE du Notebook PJ5 -1- Load&Clean ---  1.0113 seconds
